In [1]:
import json
import pandas as pd

with open('Data/WLASL_v0.3.json', "r") as file:
    data = json.load(file)
df_data = pd.json_normalize(data, record_path='instances', meta='gloss')
df_data.head()

,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id,gloss
0,"[385, 37, 885, 720]",25,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0,69241,book
1,"[190, 25, 489, 370]",25,-1,1,1,90,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0,65225,book
2,"[262, 1, 652, 480]",25,-1,1,2,110,valencia-asl,train,https://www.youtube.com/watch?v=0UsjUE-TXns,0,68011,book
3,"[123, 19, 516, 358]",25,60,1,3,113,lillybauer,train,https://www.youtube.com/watch?v=1QOYOZ3g-aY,0,68208,book
4,"[95, 0, 1180, 720]",25,-1,1,4,109,valencia-asl,train,https://www.youtube.com/watch?v=aGtIHKEdCds,0,68012,book


In [2]:
df_data.shape

(21083, 12)

In [3]:
df_misiing_video_id = pd.read_csv('Data/missing.txt', header=None, names=['video_id'])
df_misiing_video_id.head()

,video_id
0,65225
1,68011
2,68208
3,68012
4,70212


In [4]:
df_misiing_video_id

,video_id
0,65225
1,68011
2,68208
3,68012
4,70212
...,...
9098,63049
9099,63185
9100,67065
9101,63187


In [5]:
df_data['video_id'] = df_data['video_id'].astype(str).str.zfill(5)
df_misiing_video_id['video_id'] = df_misiing_video_id['video_id'].astype(str).str.zfill(5)

missing_ids_list = df_misiing_video_id['video_id'].tolist()
df_filtered = df_data[~df_data['video_id'].isin(missing_ids_list)].reset_index(drop=True)
df_filtered.shape

(11980, 12)

In [6]:
df_filtered.head()

,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id,gloss
0,"[385, 37, 885, 720]",25,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0,69241,book
1,"[462, 44, 949, 720]",25,-1,1,10,31,signschool,train,https://signstock.blob.core.windows.net/signsc...,0,07069,book
2,"[234, 17, 524, 414]",25,-1,1,17,36,startasl,train,https://s3-us-west-1.amazonaws.com/files.start...,0,07068,book
3,"[131, 26, 526, 480]",25,-1,1,22,59,asldeafined,train,https://media.asldeafined.com/vocabulary/14666...,0,07070,book
4,"[162, 54, 528, 400]",25,-1,1,24,12,aslsearch,val,http://www.aslsearch.com/signs/videos/book.mp4,0,07099,book


In [7]:
df_coded_gloss = pd.read_csv('Data/wlasl_class_list.txt', sep='\t', header=None,names=['code', 'gloss'])
df_coded_gloss.head()

,code,gloss
0,0,book
1,1,drink
2,2,computer
3,3,before
4,4,chair


In [8]:
coded_dict = dict(zip(df_coded_gloss['code'], df_coded_gloss['gloss']))

for i in range(df_filtered.shape[0]):
    for code, gloss in coded_dict.items():
        if df_filtered.loc[i, 'gloss'] == gloss:
            df_filtered.loc[i, 'gloss_encode'] = code
    if i%1000==0: print(i)

df_filtered.head()   

0


KeyboardInterrupt: 

In [9]:
coded_dict = dict(zip(df_coded_gloss['code'], df_coded_gloss['gloss']))
gloss_to_code = {v: k for k, v in coded_dict.items()}

df_filtered['gloss_encode'] = df_filtered['gloss'].map(gloss_to_code)

df_filtered.head()


,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id,gloss,gloss_encode
0,"[385, 37, 885, 720]",25,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0,69241,book,0.0
1,"[462, 44, 949, 720]",25,-1,1,10,31,signschool,train,https://signstock.blob.core.windows.net/signsc...,0,07069,book,0.0
2,"[234, 17, 524, 414]",25,-1,1,17,36,startasl,train,https://s3-us-west-1.amazonaws.com/files.start...,0,07068,book,0.0
3,"[131, 26, 526, 480]",25,-1,1,22,59,asldeafined,train,https://media.asldeafined.com/vocabulary/14666...,0,07070,book,0.0
4,"[162, 54, 528, 400]",25,-1,1,24,12,aslsearch,val,http://www.aslsearch.com/signs/videos/book.mp4,0,07099,book,0.0


In [10]:
df_filtered.to_csv('Data/data.csv', index=False)